In [2]:
from astropy.io import fits
import astropy.time as atime
import numpy as np
import astropy.units as u
import astropy.visualization as vis

from yaff.extern import solexs
from yaff import plotting as yap

import matplotlib.pyplot as plt
%matplotlib qt

import os
plt.style.use('nice.mplstyle')

## Fetch SoLEXS data from their [data server](https://pradan.issdc.gov.in/al1/protected/payload.xhtml)
SoLEXS data is served as daily `.zip` files which are further processed using `solexs-tools`.

Here we're looking at data from 2025 August 08.

We follow the procedures outlined in the [user manual](https://pradan.issdc.gov.in/al1/protected/downloadFile/solexs/SoLEXS-UserManual.pdf)
to select and prepare the data.

## Read in a light curve file for SDD2 and pick event/background intervals

In [3]:
base = "AL1_SLX_L1_20250808_v1.0/SDD2"
lc_file = "AL1_SOLEXS_20250808_SDD2_L1.lc.gz"
with fits.open(f"{base}/{lc_file}") as lcf:
    lc_dat = lcf[1]
    dt = lc_dat.header['TIMEDEL'] << u.Unit(lc_dat.header['TIMEUNIT'])
    start_time = atime.Time(lc_dat.header['TSTART'], format='unix')
    lightcurve_all_counts = lc_dat.data['COUNTS'].astype(float)
    lightcurve_time_bins = start_time + np.arange(lightcurve_all_counts.size + 1) * dt

Let's use the interval from 03:50 to 03:52, near the peak of an M class flare, for the event.

For background, let's use 03:32 to 03:33.

In [4]:
fig, ax = plt.subplots()
ax.stairs(lightcurve_all_counts, lightcurve_time_bins.datetime, color='black')

analysis_interval = atime.Time(['2025-08-08T03:50:00Z', '2025-08-08T03:52:00Z'])

ax.axvspan(
    *analysis_interval.datetime,
    color='green',
    alpha=0.4,
    label='analysis interval'
)

# Plot a segment around the interval of interest
plot_buffer = 30 << u.min
ax.set(
    yscale='log',
    ylabel='counts',
    title='solexs all counts for day',
    xlim=(
        (analysis_interval[0] - plot_buffer).datetime,
        (analysis_interval[1] + plot_buffer).datetime,
    )
)

plt.show()

## Generate the SoLEXS spectrum and response files
The [`solexs-tools`](https://pradan.issdc.gov.in/al1/protected/downloadFile/solexs/solexs_tools-1.1.tar.gz) package may be downloaded from their data distribution websitie and installed into a Python environment.
The setup is outlined in the [user manual](https://pradan.issdc.gov.in/al1/protected/downloadFile/solexs/SoLEXS-UserManual.pdf).
Install the tools before continuing.

Using the time intervals we selected before,
you can run the following `bash` snippets to slice the SoLEXS data into spectrum files.
During the slicing,
    the effective area (ARF) and energy redistribution (RMF) file paths will be printed

### Step 1: flare data
```bash
t1="2025-08-08T03:50:00"
t1=$(echo $(solexs-utc2time "$t1") | cut -d' ' -f3)

t2="2025-08-08T03:52:00"
t2=$(echo $(solexs-utc2time "$t2") | cut -d' ' -f3)

# Generate a spectrum from the L1 data files
spec_fn="AL1_SLX_L1_20250808_v1.0/SDD2/AL1_SOLEXS_20250808_SDD2_L1.pi.gz"
gti_fn="AL1_SLX_L1_20250808_v1.0/SDD2/AL1_SOLEXS_20250808_SDD2_L1.gti.gz"

solexs-genspec -i "$spec_fn" \
    -tstart "$t1" -tstop "$t2"\
    -gti "$gti_fn"
```

### Step 2: background data
```bash
t1="2025-08-08T02:32:00"
t1=$(echo $(solexs-utc2time "$t1") | cut -d' ' -f3)

t2="2025-08-08T03:33:00"
t2=$(echo $(solexs-utc2time "$t2") | cut -d' ' -f3)

# Generate a spectrum from the L1 data files
spec_fn="AL1_SLX_L1_20250808_v1.0/SDD2/AL1_SOLEXS_20250808_SDD2_L1.pi.gz"
gti_fn="AL1_SLX_L1_20250808_v1.0/SDD2/AL1_SOLEXS_20250808_SDD2_L1.gti.gz"

solexs-genspec -i "$spec_fn" \
    -tstart "$t1" -tstop "$t2"\
    -gti "$gti_fn"
```

In [5]:
data_fn = 'AL1_SOLEXS_20250808_SDD2_L1_225000_225200.pi'
with fits.open(data_fn) as dat:
    cts = np.array(dat[1].data['COUNTS'], dtype=int) << u.ct

    # Assume Poisson error, then add on systematics
    sys_uncert = 0.04
    err = np.sqrt(cts.to_value(u.ct))
    err = np.sqrt(err**2 + (sys_uncert * cts.to_value(u.ct))**2) << u.ct

bkg_fn = 'AL1_SOLEXS_20250808_SDD2_L1_213200_223300.pi'
with fits.open(bkg_fn) as dat:
    bkg_cts = np.array(dat[1].data['COUNTS'], dtype=int) << u.ct

    # Assume Poisson error, then add on systematics
    sys_uncert = 0.04
    bkg_err = np.sqrt(cts.to_value(u.ct))
    bkg_err = np.sqrt(bkg_err**2 + (sys_uncert * cts.to_value(u.ct))**2) << u.ct

data_exposure = 5 << u.min
bkg_exposure = 1 << u.min

In [6]:
# Now, load in the response data, using the paths output by the SoLEXS tools

arf_fn = 'solexs_tools-1.1/CALDB/arf/solexs_arf_SDD2_v1.arf'
rmf_fn = 'solexs_tools-1.1/CALDB/response/rmf/solexs_gaussian_SDD2_v1.rmf'

# Put the arf and rmf data into one dict,
# but keep the information separated by keys
response_data = (
    solexs.read_arf(arf_fn) |
    solexs.read_rmf(rmf_fn)
)

## Now that the data and response matrices have been loaded, let's define our models.

In [ ]:
from yaff import common_models, common_likelihoods
from yaff import fitting
import importlib
_ = importlib.reload(common_models)

In [ ]:
from sunkit_spex.legacy import thermal
import importlib
importlib.reload(thermal)
import copy

default_abun_type = 'sun_coronal_ext'
atomic_numbers = {
    'mg': 12, 'al': 13,
    'si': 14, 's': 16,
    'ar': 18, 'ca': 20,
    'fe': 26
}
# atomic_numbers = {v: k for (k, v) in atomic_numbers.items()}

def thermal_with_relative_abundances(args: dict[str, object]):
    energies: np.ndarray = args['photon_energy_edges']
    params: dict[str, fitting.Parameter] = args['parameters']

    # For every atomic number given,
    # scale the default abundances by the coronal values.
    rel_abuns = list()
    for (k, v) in params.items():
        if k not in atomic_numbers:
            continue
        rel_abuns.append((atomic_numbers[k], v.value))

    return thermal.thermal_emission(
        energies << u.keV,
        temperature=params['temperature'].as_quantity(),
        emission_measure=params['emission_measure'].as_quantity(),
        relative_abundances=rel_abuns
    ).to_value(u.ph / u.cm**2 / u.keV / u.s)

In [ ]:
# Cut out any photon bins < 1.5 keV
# (thermal emission function doesn't like it)
ph_bins = response_data['photon_energy_bins']
mids = ph_bins[:-1] + np.diff(ph_bins) / 2

limit = 2.5 << u.keV
keep = (mids > limit)
keep_edges = (ph_bins >= limit)

dp = fitting.DataPacket(
    counts=cts << u.ct,
    counts_error=err << u.ct,
    background_counts=0 * cts,
    background_counts_error=0 * cts,
    effective_exposure=data_exposure,
    count_energy_edges=response_data['count_energy_bins'],
    photon_energy_edges=response_data['photon_energy_bins'][keep_edges],
    response_matrix=(response_data['effective_area'][keep] * response_data['redistribution_matrix'][:, keep])
)

In [ ]:
params = {
    'temperature': fitting.Parameter(20 << u.MK, frozen=False),
    'emission_measure': fitting.Parameter(0.5 << (1e49 * u.cm**-3), frozen=False),
}

log_priors = {
    'temperature': fitting.simple_bounds(4, 30),
    'emission_measure': fitting.simple_bounds(1e-3, 1e3)
}

# There are a lot of elements we can vary, so add them with loops separately
# elemental_params = ['mg', 'si', 's', 'ar', 'ca', 'fe']
elemental_params = ['fe', 'ca', 'ar', 'si', 's']
for n in elemental_params:
    # For the first round, fix the abundances and just work on fitting the continuum.
    params[n] = fitting.Parameter(1. << u.one, frozen=False)
    log_priors[n] = fitting.simple_bounds(0.1, 3.0)

lowe, highe = 3, 12
mids = dp.count_energy_mids
likelihood = common_likelihoods.chi_squared_factory(
    restriction=(fit_restriction := (lowe < mids) & (mids < highe))
)

fr = fitting.BayesFitterWithGain(
    data=dp,
    model_function=thermal_with_relative_abundances,
    parameters=params,
    log_priors=log_priors,
    log_likelihood=likelihood,
)

In [ ]:
fr.parameters['gain_slope'].frozen = False
fr.parameters['gain_offset'].frozen = False
fr.parameters

In [ ]:
fr = fitting.levenberg_minimize(
    fr,
    restriction=fit_restriction,
    jac='3-point',
    ftol=1e-6
)

In [ ]:
# Freeze the gain params so they do not change later on
fr.parameters['gain_offset'].frozen = True
fr.parameters['gain_slope'].frozen = True
fr.parameters

In [ ]:
fig = plt.figure()
ret = yap.plot_data_model(fr, fig=fig)
ret['data_ax'].axvspan(lowe, highe, color='red', alpha=0.1, zorder=-1)

mod = fr.eval_model()
ret['data_ax'].legend()
plt.show()

In [ ]:
fr.run_emcee(
    emcee_constructor_kw={'nwalkers': 20},
    emcee_run_kw={'nsteps': 1000}
)

In [ ]:
samples = fr.generate_model_samples(100)
yap.plot_data_model(fr, samples)
plt.show()

In [ ]:
yap.plot_parameter_chains(
    fr,
    fr.free_param_names,
    list(fr.free_parameters.values())
)